# Formal Ontology of Mathematics Notebook

## Purpose
This notebook aims to build a knowledge graph (KG) representing the formal ontology of mathematics, specifically focusing on Euclid's Elements, Book 1. It extracts information from various input files (CSV and TXT) containing definitions, postulates, common notions, and concepts from Euclid's Elements, Book 1.

## Functionality
1. **Data Ingestion**: It reads data from input files like "ontology_definitions_concepts.txt", "Euclid.Postulates.Book1.csv", and "Euclid.CommonNotions.Book1.csv".
2. **Knowledge Representation**: It utilizes the RDFLib library to create and populate a knowledge graph, representing entities and their relationships using triples (subject, predicate, object).
3. **Ontology Structure**: The KG follows a defined ontology with classes like Concept, Postulate, Common Notion, Operation Type, and Relation Type.
4. **Relationship Modeling**: It establishes relationships between entities using object properties like refers_to, is_used_in, has_definition, has_subject, has_domain, etc.
5. **Data Processing**: Functions like `add_postulates`, `add_common_notions`, `add_statement`, `add_concept_hierarchy` process and structure the data to populate the knowledge graph.
6. **Ontology Output**: It exports the generated KG in Turtle format into the "output" directory as "euclid_book1.ttl".

## Libraries Used
- **RDFLib**: for creating and manipulating RDF graphs.
- **Pandas**: for data handling and manipulation.
- **OS**: for interacting with the operating system, such as mounting Google Drive and changing directories.
- **re**: for regular expressions.
- **Google Colab**: for accessing and mounting Google Drive.


## Workflow
The notebook first initializes an empty RDF graph. It then defines a set of core ontology items (classes and object properties). It loads data from the input files and adds them as triples to the RDF graph. The process includes specific functions for adding postulates, common notions, concepts, and their relationships. Finally, the notebook outputs the populated knowledge graph in Turtle format.

## Assumptions
- Assumes the input files are present in the specified directory ("input").
- Assumes the required libraries are installed.
- Assumes the Google Colab environment is used for execution.

In [ ]:
%pip install rdflib

In [ ]:
import google
import os

# prompt: mount drive here to read files from the folder "My Drive > Colab_Notebooks > Formal_Ontology_of_Mathematics"
google.colab.drive.mount('/content/drive')

os.chdir("/content/drive/My Drive/Colab_Notebooks/Formal_Ontology_of_Mathematics")
!ls

In [ ]:
import pandas as pd
import os
import re
import rdflib

import modules.utils as utils
import modules.tbox as tbox
import modules.concepts as concepts
import modules.postulates_module as postulate_module
import modules.common_notions_module as common_notions_module
import modules.propositions_module as propositions_module
import modules.concepts_module as concepts_module

# common IRIs
rdf_type = rdflib.RDF.type
rdfs_label = rdflib.RDFS.label
rdfs_subclassof = rdflib.RDFS.subClassOf
rdfs_range = rdflib.RDFS.range
skos_prefLabel = rdflib.SKOS.prefLabel
skos_altLabel = rdflib.SKOS.altLabel
owl_class = rdflib.OWL.Class
owl_individual = rdflib.OWL.NamedIndividual
owl_object_property = rdflib.OWL.ObjectProperty
owl_data_property = rdflib.OWL.DatatypeProperty
owl_annotation_property = rdflib.OWL.AnnotationProperty
xsd_boolean = rdflib.XSD.boolean
xsd_true = rdflib.Literal("true", datatype=xsd_boolean)
xsd_false = rdflib.Literal("false", datatype=xsd_boolean)

# classes IRIs
common_notion_class = utils.create_iri("Common notion", namespace="https://www.foom.com/core")
concept_class = utils.create_iri("Concept", namespace="https://www.foom.com/core")
concept_type_class = utils.create_iri("Concept type", namespace="https://www.foom.com/core")
gist_class = utils.create_iri("Gist", namespace="https://www.foom.com/core")
enumeration_class = utils.create_iri("Enumeration", namespace="https://www.foom.com/core")
implication_class = utils.create_iri("Implication", namespace="https://www.foom.com/core")
magnitude_class = utils.create_iri("Magnitude", namespace="https://www.foom.com/core")
moral_class = utils.create_iri("Moral", namespace="https://www.foom.com/core")
operation_type_class = utils.create_iri("Operation type", namespace="https://www.foom.com/core")
operation_instance_class = utils.create_iri("Operation instance", namespace="https://www.foom.com/core")
proposition_class = utils.create_iri("Proposition", namespace="https://www.foom.com/core")
proposition_type_class = utils.create_iri("Proposition type", namespace="https://www.foom.com/core")
relation_type_class = utils.create_iri("Relation type", namespace="https://www.foom.com/core")
relation_instance_class = utils.create_iri("Relation instance", namespace="https://www.foom.com/core")
set_class = utils.create_iri("Set", namespace="https://www.foom.com/core")
statement_class = utils.create_iri("Statement", namespace="https://www.foom.com/core")

# individual IRIs
elements_book_1 = rdflib.URIRef("https://www.foom.com/core#document__elements_book_1")
object_individual = utils.create_iri("Concept type: Object", namespace="https://www.foom.com/core")
relation_individual = utils.create_iri("Concept type: Relation", namespace="https://www.foom.com/core")
operation_individual = utils.create_iri("Concept type: Operation", namespace="https://www.foom.com/core")

# object properties IRIs
elements_book_1 = utils.create_iri("Document: Elements Book 1", namespace="https://www.foom.com/core")
proposition_type_construction = utils.create_iri("Proposition type: Construction", namespace="https://www.foom.com/core")
proposition_type_theorem = utils.create_iri("Proposition type: Theorem", namespace="https://www.foom.com/core")

refers_to = utils.create_iri("refers to", namespace="https://www.foom.com/core")
definition_refers_to = utils.create_iri("definition refers to", namespace="https://www.foom.com/core")

has_conceptual_component = utils.create_iri("has conceptual component", namespace="https://www.foom.com/core")
is_conceptual_component_of = utils.create_iri("is conceptual component of", namespace="https://www.foom.com/core")

is_sub_concept_of = utils.create_iri("is sub-concept of", namespace="https://www.foom.com/core")
is_super_concept_of = utils.create_iri("is super-concept of", namespace="https://www.foom.com/core")

is_used_in_definition_of = utils.create_iri("is used in definition of", namespace="https://www.foom.com/core")
is_defined_in = utils.create_iri("is defined in", namespace="https://www.foom.com/core")

contains_definition_of = utils.create_iri("contains_definition_of", namespace="https://www.foom.com/core")
is_used_in = utils.create_iri("is used in", namespace="https://www.foom.com/core")

has_definition = utils.create_iri("has definition", namespace="https://www.foom.com/core")
defines = utils.create_iri("defines", namespace="https://www.foom.com/core")

has_subject = utils.create_iri("has subject", namespace="https://www.foom.com/core")
has_predicate = utils.create_iri("has predicate", namespace="https://www.foom.com/core")
has_object = utils.create_iri("has object", namespace="https://www.foom.com/core")

has_domain = utils.create_iri("has domain", namespace="https://www.foom.com/core")
is_domain_of = utils.create_iri("is domain of", namespace="https://www.foom.com/core")
has_range = utils.create_iri("has_range", namespace="https://www.foom.com/core")
is_range_of = utils.create_iri("is range of", namespace="https://www.foom.com/core")

has_statement = utils.create_iri("has statement", namespace="https://www.foom.com/core")
is_statement_of = utils.create_iri("is statement of", namespace="https://www.foom.com/core")

has_implication = utils.create_iri("has implication", namespace="https://www.foom.com/core")
is_implication_of = utils.create_iri("is implication of", namespace="https://www.foom.com/core")

is_in = utils.create_iri("is in", namespace="https://www.foom.com/core")
contains = utils.create_iri("contains", namespace="https://www.foom.com/core")

has_given_concept = utils.create_iri("has given concept", namespace="https://www.foom.com/core")
is_given_concept_of = utils.create_iri("is given concept of", namespace="https://www.foom.com/core")

has_gist = utils.create_iri("has gist", namespace="https://www.foom.com/core")
is_gist_of = utils.create_iri("is gist of", namespace="https://www.foom.com/core")

has_moral = utils.create_iri("has moral", namespace="https://www.foom.com/core")
is_moral_of = utils.create_iri("is moral of", namespace="https://www.foom.com/core")

has_concept_type = utils.create_iri("has concept type", namespace="https://www.foom.com/core")
is_concept_type_of = utils.create_iri("is concept type of", namespace="https://www.foom.com/core")

In [ ]:
# initialize graph
kg = rdflib.Graph()

# tbox and foundational triples
ontology_items = {
    # classes
    ("Concept", owl_class),
    ("Concept type", owl_class),
    ("Common notion", owl_class),
    ("Definition", owl_class),
    ("Document", owl_class),
    ("Enumeration", owl_class),
    ("Gist", owl_class),
    ("Implication", owl_class),
    ("Magnitude", owl_class),
    ("Moral", owl_class),
    ("Operation type", owl_class),
    ("Operation instance", owl_class),
    ("Postulate", owl_class),
    ("Proposition", owl_class),
    ("Proposition type", owl_class),
    ("Relation type", owl_class),
    ("Relation instance", owl_class),
    ("Set", owl_class),
    ("Statement", owl_class),

    # object properties
    ("refers to", owl_object_property),
    ("definition refers to", owl_object_property),

    ("has conceptual component", owl_object_property),
    ("is conceptual component of", owl_object_property),

    ("is sub-concept of", owl_object_property),
    ("is super-concept of", owl_object_property),

    ("is used in definition of", owl_object_property),
    ("is defined in", owl_object_property),

    ("contains_definition_of", owl_object_property),
    ("is used in", owl_object_property),

    ("has definition", owl_object_property),
    ("defines", owl_object_property),

    ("has subject", owl_object_property),
    ("has predicate", owl_object_property),
    ("has object", owl_object_property),

    ("has domain", owl_object_property),
    ("is domain of", owl_object_property),
    ("has range", owl_object_property),
    ("is range of", owl_object_property),

    ("has statement", owl_object_property),
    ("is statement of", owl_object_property),

    ("has implication", owl_object_property),
    ("is implication of", owl_object_property),

    ("is in", owl_object_property),
    ("contains", owl_object_property),

    ("has given concept", owl_object_property),
    ("is given concept of", owl_object_property),

    ("has gist", owl_object_property),
    ("is gist of", owl_object_property),

    ("has moral", owl_object_property),
    ("is moral of", owl_object_property),

    ("has concept type", owl_object_property),
    ("is concept type of", owl_object_property)
}
triples = {
    (elements_book_1, rdf_type, owl_individual),
    (elements_book_1, rdfs_label, rdflib.Literal("Document: Elements Book 1")),
    (elements_book_1, skos_prefLabel, rdflib.Literal("Elements Book 1")),
    (elements_book_1, rdf_type, utils.create_iri("Document", namespace="https://www.foom.com/core")),

    (utils.create_iri("Proposition type", namespace="https://www.foom.com/core"), rdfs_subclassof, enumeration_class),
    (proposition_type_construction, rdf_type, owl_individual),
    (proposition_type_construction, rdf_type, proposition_type_class),
    (proposition_type_construction, rdfs_label, rdflib.Literal("Proposition type: Construction")),
    (proposition_type_construction, skos_prefLabel, rdflib.Literal("Construction")),
    (proposition_type_theorem, rdf_type, owl_individual),
    (proposition_type_theorem, rdf_type, proposition_type_class),
    (proposition_type_theorem, rdfs_label, rdflib.Literal("Proposition type: Theorem")),
    (proposition_type_theorem, skos_prefLabel, rdflib.Literal("Theorem")),

    (concept_type_class, rdfs_subclassof, enumeration_class)
}

kg = tbox.add_tbox(kg, ontology_items, triples)

# abox: concepts
concepts_input_file_path = "input/ontology_definitions_concepts.txt"
kg = concepts.main_add_definition_concepts(concepts_input_file_path, kg)

# abox: postulates
postulates_input_file_path = "input/Euclid.Postulates.Book1.csv"
kg = postulate_module.add_postulates(kg, postulates_input_file_path)

# abox: add common notions
common_notions_input_file_path = "input/Euclid.CommonNotions.Book1.csv"
kg = common_notions_module.add_common_notions(kg, common_notions_input_file_path)

# abox: add propositions
propositions_input_file_path = "input/Euclid.Propositions.Book1.csv"
kg = propositions_module.add_propositions(kg, propositions_input_file_path)

# abox: add concepts with hierarchy and type
concepts_analysis_input_file_path = "input/Euclid.ConceptsAnalysis.Book1.csv"
kg = concepts_module.add_concepts(kg, concepts_analysis_input_file_path)

# output ontology
utils.output_ontology(kg, "output", "euclid_book1.ttl", "turtle")

In [ ]:
def add_concepts(kg: rdflib.Graph,
                 concepts_analysis_input_file_path: str) -> rdflib.Graph:
    # add concept types
    kg = add_concept_types(kg, {"Object", "Operation", "Relation"})

    # read concepts data
    concepts_df = pd.read_csv(concepts_analysis_input_file_path).fillna("")
    for i in concepts_df.index:
        concept_preflabel = concepts_df.at[i, "concepts"].replace("_", " ").strip().capitalize()

        # add concept
        concept_iri = utils.create_iri(f"Concept: {concept_preflabel}", namespace="https://www.foom.com/core")

        # add concept
        kg = concepts.add_triples(kg, concept_preflabel, concept_iri, concept_class, "Concept")

        # add hierarchy
        concept_hierarchy_preflabel = concepts_df.at[i, "hierarchy"].replace("_", " ").strip().capitalize()
        if concept_hierarchy_preflabel:
            kg = add_concept_hierarchy(kg, concept_iri, concept_hierarchy_preflabel)

        # add concept type
        concept_type_iri = utils.create_iri(f"Concept type: {concepts_df.at[i, 'type']}", namespace="https://www.foom.com/core")
        kg.add((concept_iri, has_concept_type, concept_type_iri))
        kg.add((concept_type_iri, is_concept_type_of, concept_iri))

        # add datatype properties
        datatype_properties = [datatype_property.strip() for datatype_property in concepts_df.at[i, "datatype_property"].split(",")]
        for datatype_property in datatype_properties:
            datatype_property_iri = utils.create_iri(datatype_property, namespace="https://www.foom.com/core")
            kg = add_datatype_property(kg, datatype_property, datatype_property_iri)
            kg.add((concept_iri, datatype_property_iri, xsd_true))

    return kg

def add_datatype_property(kg: rdflib.Graph,
                          datatype_property_preflabel: str,
                          datatype_property_iri = rdflib.URIRef) -> rdflib.Graph:
    kg.add((datatype_property_iri, rdf_type, owl_data_property))
    kg.add((datatype_property_iri, rdfs_label, rdflib.Literal(datatype_property_preflabel)))
    kg.add((datatype_property_iri, skos_prefLabel, rdflib.Literal(datatype_property_preflabel)))
    kg.add((datatype_property_iri, rdfs_range, xsd_boolean))
    return kg

def add_concept_types(kg: rdflib.Graph,
                      concept_types_preflabels: set) -> rdflib.Graph:
    for concept_type_preflabel in concept_types_preflabels:
        concept_type_iri = utils.create_iri(f"Concept type: {concept_type_preflabel}", namespace="https://www.foom.com/core")
        kg = concepts.add_triples(kg, concept_type_preflabel, concept_type_iri, concept_type_class, "Concept type")
    return kg

def add_concept_hierarchy(kg: rdflib.Graph,
                          concept_iri: rdflib.URIRef,
                          concept_hierarchy_preflabel: str) -> rdflib.Graph:
    concept_hierarchy_iri = utils.create_iri(f"Concept: {concept_hierarchy_preflabel}", namespace="https://www.foom.com/core")
    kg = concepts.add_triples(kg, concept_hierarchy_preflabel, concept_hierarchy_iri, concept_class, "Concept")
    kg.add((concept_iri, is_sub_concept_of, concept_hierarchy_iri))
    kg.add((concept_hierarchy_iri, is_super_concept_of, concept_iri))
    return kg

concepts_analysis_input_file_path = "input/Euclid.ConceptsAnalysis.Book1.csv"
kg = add_concepts(kg, concepts_analysis_input_file_path)
# output ontology
utils.output_ontology(kg, "output", "euclid_book1.ttl", "turtle")